# RESEARCH NOTEBOOK --> RAJ_REVERSION

In [ ]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [ ]:
import pandas as pd
import pandas_ta as ta  # noqa: F401
from core.data_sources import CLOBDataSource

# Initialize the data source
clob = CLOBDataSource()

In [ ]:
# Define the parameters
exchange = "binance_perpetual"
trading_pair = "DOT-USDT"
timeframe = "1h"
days = 120

In [ ]:
# Get the candles
candles = await clob.get_candles_last_days(
    exchange, trading_pair, timeframe, days, from_trades=False)

In [ ]:
candles.plot(type="candles")

In [ ]:
import numpy as np
import pandas as pd

import numpy as np

def calculate_pivots(df: pd.DataFrame, source_column: str, left: int, right: int):
    """
    Calculate pivot highs and lows based on a source column.
    
    Args:
        df (pd.DataFrame): DataFrame containing the price data
        source_column (str): Column name to use for pivot calculations
        left (int): Number of bars to look left
        right (int): Number of bars to look right
    
    Returns:
        tuple: (pivot_highs_idx, pivot_highs, pivot_lows_idx, pivot_lows)
            - pivot_highs_idx: List of datetime indices where pivot highs occur
            - pivot_highs: List of high prices at pivot high points
            - pivot_lows_idx: List of datetime indices where pivot lows occur
            - pivot_lows: List of low prices at pivot low points
    """
    pivot_highs_index = []
    pivot_lows_index = []
    
    # Find pivot points
    for i, value in enumerate(df[source_column]):
        left_index = max(0, i-left)
        right_index = min(len(df), i+right+1)
        range_values = df.iloc[left_index:right_index][source_column]
        
        if value == range_values.max():
            pivot_highs_index.append(i)
        if value == range_values.min():
            pivot_lows_index.append(i)
    
    # Convert indices to datetime and get corresponding prices
    pivot_highs_idx = [df.iloc[i].name for i in pivot_highs_index]
    pivot_lows_idx = [df.iloc[i].name for i in pivot_lows_index]
    pivot_highs = [df.iloc[i][source_column] for i in pivot_highs_index]
    pivot_lows = [df.iloc[i][source_column] for i in pivot_lows_index]
    
    return pivot_highs_idx, pivot_highs, pivot_lows_idx, pivot_lows


def calculate_alma(series: pd.Series, window_size: int = 9, offset: float = 0.85, sigma: float = 6) -> pd.Series:
    """
    Calculate Arnaud Legoux Moving Average (ALMA)
    
    Args:
        series (pd.Series): Input price series
        window_size (int): The window size for the moving average
        offset (float): Controls the smoothing (from 0 to 1)
        sigma (float): Controls the smoothing width
    
    Returns:
        pd.Series: ALMA values
    """
    # Initialize the result series
    result = pd.Series(index=series.index, dtype=float)
    
    # Calculate the offset point
    m = offset * (window_size - 1)
    # Calculate s
    s = window_size / sigma
    
    # Calculate weights
    weights = np.zeros(window_size)
    for i in range(window_size):
        weights[i] = np.exp(-1 * (i - m) ** 2 / (2 * s ** 2))
    
    # Normalize weights
    weights = weights / weights.sum()
    
    # Calculate ALMA
    for i in range(window_size - 1, len(series)):
        window = series.iloc[i - window_size + 1:i + 1]
        result.iloc[i] = (window * weights).sum()
    
    return result


In [ ]:
# FEATURES

# Mean Reversion Strategy Parameters
close_alma_length = 80  # Length input 
close_alma_offset = 0.85  # Offset input
close_alma_sigma = 16  # Sigma input

# Pivot Parameters
left = 7  # Left pivot input
right = 7  # Right pivot input
array_percent = 86  # Percentile threshold
rolling_window = 200


# ALMA over mean Parameters
diff_alma_length = 9  # Second ALMA length
diff_alma_offset = 0.85  # Second ALMA offset  
diff_alma_sigma = 16  # Second ALMA sigma

# Add indicators
candles_df = candles.data

# Calculate ALMA
candles_df["alma"] = calculate_alma(candles_df["close"], window_size=close_alma_length, offset=close_alma_offset, sigma=close_alma_sigma)

# Calculate source based on conditions
candles_df["src"] = np.where(
    ((candles_df["high"] - candles_df["alma"]) > abs(candles_df["low"] - candles_df["alma"])) & 
    ((candles_df["high"] - candles_df["open"]) > abs(candles_df["low"] - candles_df["open"])),
    candles_df["high"],
    np.where(
        ((candles_df["high"] - candles_df["alma"]) < abs(candles_df["low"] - candles_df["alma"])) &
        ((candles_df["high"] - candles_df["open"]) < abs(candles_df["low"] - candles_df["open"])),
        candles_df["low"],
        candles_df["close"]
    )
)

# Calculate percentage difference from ALMA using the source
candles_df["diff"] = 100 * (candles_df["src"] - candles_df["alma"]) / candles_df["alma"]

# Alma over mean
candles_df["alma_over_mean"] = calculate_alma(candles_df["diff"], window_size=diff_alma_length, offset=diff_alma_offset, sigma=diff_alma_sigma)
# Calculate Pivots

pivot_highs_idx, pivot_highs, pivot_lows_idx, pivot_lows = calculate_pivots(
    df=candles_df,
    source_column="diff",
    left=left,
    right=right
)

# OLD PIVOT WAY
container = pivot_highs + pivot_lows
pct_rank = np.percentile(container, array_percent)

# NEW PIVOT WAY
pivot_series = pd.Series(index=candles_df.index, dtype=float)
pivot_series.loc[pivot_highs_idx] = pivot_highs
pivot_series.loc[pivot_lows_idx] = pivot_lows

# Calculate rolling percentile (using last 100 periods by default)
candles_df["pct_rank"] = (
    pivot_series.rolling(window=rolling_window, min_periods=1)
    .apply(lambda x: np.percentile(x.dropna(), array_percent))
    .fillna(method='ffill')
)

candles_df["cross_over_alma_diff"] = (candles_df["diff"] > candles_df["alma_over_mean"]) & (candles_df["diff"].shift(1) < candles_df["alma_over_mean"].shift(1))
candles_df["cross_under_alma_diff"] = (candles_df["diff"] < candles_df["alma_over_mean"]) & (candles_df["diff"].shift(1) > candles_df["alma_over_mean"].shift(1))
long_condition = candles_df["cross_over_alma_diff"] & (candles_df["diff"] < -candles_df["pct_rank"])
short_condition = candles_df["cross_under_alma_diff"] & (candles_df["diff"] > candles_df["pct_rank"])

candles_df["signal"] = 0
candles_df.loc[long_condition, "signal"] = 1
candles_df.loc[short_condition, "signal"] = -1

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03, 
                    subplot_titles=(trading_pair, 'Pivot Highs and Lows'),
                    row_heights=[0.7, 0.3])
# plot the pivot highs and lows over the candles chart
fig.add_trace(candles.candles_trace(), row=1, col=1)
# add alma and src over candles
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df["alma"], mode="lines", line=dict(color="lime", width=2), name="ALMA"), row=1, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df["src"], mode="lines", line=dict(color="violet", width=1), name="SRC"), row=1, col=1)
# add signals as green and red triangles
long_signals = candles_df[candles_df["signal"] == 1]
short_signals = candles_df[candles_df["signal"] == -1]
fig.add_trace(go.Scatter(x=long_signals.index, y=long_signals["close"], mode="markers", marker=dict(color="green", size=10, symbol="triangle-up"), name="Long"), row=1, col=1)
fig.add_trace(go.Scatter(x=short_signals.index, y=short_signals["close"], mode="markers", marker=dict(color="red", size=10, symbol="triangle-down"), name="Short"), row=1, col=1)


# plot over the second subplot
# Add diff as a white line, alma over mean as an orange line and add to green hlines for PCT Rank and -PCT Rank and the pivot highs and lows as red and green markers
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df["diff"], mode="lines", line=dict(color="white", width=2), name="Diff"), row=2, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df["alma_over_mean"], mode="lines", line=dict(color="orange", width=2), name="ALMA Over Mean"), row=2, col=1)

# example: fig.add_hline(row["sniper_upper_price"], line_dash="dot", line_color="red")
fig.add_hline(y=pct_rank, line_dash="dot", line_color="green", row=2, col=1)
fig.add_hline(y=-pct_rank, line_dash="dot", line_color="green", row=2, col=1)
fig.add_trace(go.Scatter(x=pivot_highs_idx, y=pivot_highs, mode="markers", marker=dict(color="red", size=10), name="Pivot Highs"), row=2, col=1)
fig.add_trace(go.Scatter(x=pivot_lows_idx, y=pivot_lows, mode="markers", marker=dict(color="green", size=10), name="Pivot Lows"), row=2, col=1)

# Plot new PCT Rank
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df["pct_rank"], mode="lines", line=dict(color="blue", width=2), name="PCT Rank"), row=2, col=1)

# remove range slider and add dark theme and remove background grid
fig.update_layout(xaxis_rangeslider_visible=False,
                  height=800,
                  width=1500,
                  plot_bgcolor='#1e1e1e',
                  paper_bgcolor='#1e1e1e',
                  font=dict(color='white'),
                  xaxis=dict(showgrid=False),
                  yaxis=dict(showgrid=False),
                  )
fig.update_layout(title=f'{exchange} - {trading_pair} - {timeframe}')
fig.show()


In [ ]:
# Compute the signal
# Long:
# - Diff cross over ALMA Diff
# - Diff < -PCT Rank

# Short:
# - Diff cross under ALMA Diff
# - Diff > PCT Rank








